In [1]:
from dialogue2graph.metrics.no_llm_metrics import all_utterances_present, dialogues_are_valid_paths
from dialogue2graph.metrics.llm_metrics import compare_graphs
from dialogue2graph.pipelines.core.graph import Graph
from dialogue2graph.pipelines.core.dialogue import Dialogue

/home/peshkichev/miniconda3/envs/py311/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(


In [2]:
from pydantic_settings import BaseSettings, SettingsConfigDict
from typing import Optional

In [3]:
class EnvSettings(BaseSettings, case_sensitive=True):

    model_config = SettingsConfigDict(env_file="./.env", env_file_encoding="utf-8")

    OPENAI_API_KEY: Optional[str]
    OPENAI_BASE_URL: Optional[str]
    HUGGINGFACE_TOKEN: Optional[str]
    EMBEDDER_DEVICE: Optional[str]


env_settings = EnvSettings()

In [4]:
from datasets import load_dataset

In [5]:
dataset = load_dataset("DeepPavlov/d2g_generated", token=env_settings.HUGGINGFACE_TOKEN)

In [6]:
data = dataset['train'][0]

In [7]:
dataset['train'][0]['dialogues']

[{'id': 'Responding to DMs on Instagram/Facebook._1_0',
  'messages': [{'participant': 'assistant',
    'text': 'Hi there! How can I assist you today?'},
   {'participant': 'user', 'text': 'I want to place an order.'},
   {'participant': 'assistant',
    'text': 'Great! What product are you interested in?'},
   {'participant': 'user', 'text': 'I’d like a t-shirt.'},
   {'participant': 'assistant', 'text': 'What size would you like?'},
   {'participant': 'user', 'text': 'Medium, please.'},
   {'participant': 'assistant', 'text': 'Would you like that in red or blue?'},
   {'participant': 'user', 'text': 'Red, please.'},
   {'participant': 'assistant',
    'text': 'Perfect! Can I have your shipping address?'},
   {'participant': 'user', 'text': 'Sure, it’s 123 Main St.'},
   {'participant': 'assistant',
    'text': 'Thank you! Your order has been placed. Is there anything else I can help you with?'},
   {'participant': 'user', 'text': 'Actually, can I change my order?'},
   {'participant'

In [8]:
dialogues = [Dialogue.from_list(c['messages'], c['id']) for c in data['dialogues']]

In [9]:
dialogues[0]

Dialogue(messages=[DialogueMessage(text='Hi there! How can I assist you today?', participant='assistant'), DialogueMessage(text='I want to place an order.', participant='user'), DialogueMessage(text='Great! What product are you interested in?', participant='assistant'), DialogueMessage(text='I’d like a t-shirt.', participant='user'), DialogueMessage(text='What size would you like?', participant='assistant'), DialogueMessage(text='Medium, please.', participant='user'), DialogueMessage(text='Would you like that in red or blue?', participant='assistant'), DialogueMessage(text='Red, please.', participant='user'), DialogueMessage(text='Perfect! Can I have your shipping address?', participant='assistant'), DialogueMessage(text='Sure, it’s 123 Main St.', participant='user'), DialogueMessage(text='Thank you! Your order has been placed. Is there anything else I can help you with?', participant='assistant'), DialogueMessage(text='Actually, can I change my order?', participant='user'), DialogueMe

In [7]:
all_utterances_present(Graph(data['graph']),dialogues)

Graph has all the dialogues


True

In [10]:
dialogues_are_valid_paths(Graph(data['graph']),dialogues)

[{'value': True,
  'description': 'graph has all paths from Dialogue Responding to DMs on Instagram/Facebook._1_0'},
 {'value': True,
  'description': 'graph has all paths from Dialogue Responding to DMs on Instagram/Facebook._1_1'},
 {'value': True,
  'description': 'graph has all paths from Dialogue Responding to DMs on Instagram/Facebook._1_2'},
 {'value': True,
  'description': 'graph has all paths from Dialogue Responding to DMs on Instagram/Facebook._1_3'},
 {'value': True,
  'description': 'graph has all paths from Dialogue Responding to DMs on Instagram/Facebook._1_4'},
 {'value': True,
  'description': 'graph has all paths from Dialogue Responding to DMs on Instagram/Facebook._1_5'}]

In [8]:
data['graph']['edges'] += [{'source': 9,
    'target': 2,
    'utterances': ['I want to cancel my order.']}]

In [47]:
all_utterances_present(Graph(data['graph']),dialogues)

absent in graph:  set()
absent in dialogues:  {'i want to cancel my order.'}
Graph has all the dialogues


False

In [11]:
data['graph']['edges'].pop(0)

{'source': 1, 'target': 2, 'utterances': ['I want to place an order.']}

In [49]:
all_utterances_present(Graph(data['graph']),dialogues)

absent in graph:  {'i want to place an order.'}
absent in dialogues:  {'i want to cancel my order.'}


False

In [12]:
dialogues_are_valid_paths(Graph(data['graph']),dialogues)

[{'value': False,
  'description': "graph has no connection between next pairs [('Hi there! How can I assist you today?', 'I want to place an order.'), ('I want to place an order.', 'Great! What product are you interested in?')] from Dialogue Responding to DMs on Instagram/Facebook._1_0"},
 {'value': False,
  'description': "graph has no connection between next pairs [('Hi there! How can I assist you today?', 'I want to place an order.'), ('I want to place an order.', 'Great! What product are you interested in?')] from Dialogue Responding to DMs on Instagram/Facebook._1_1"},
 {'value': False,
  'description': "graph has no connection between next pairs [('Hi there! How can I assist you today?', 'I want to place an order.'), ('I want to place an order.', 'Great! What product are you interested in?')] from Dialogue Responding to DMs on Instagram/Facebook._1_2"},
 {'value': True,
  'description': 'graph has all paths from Dialogue Responding to DMs on Instagram/Facebook._1_3'},
 {'value':

In [21]:
data_2 = dataset['train'][1]

In [22]:
compare_graphs(Graph(data['graph']),Graph(data_2['graph']))

{'value': False, 'description': 'Numbers of nodes do not match: 9 != 8'}